## Thư viện

In [1]:
import numpy as np
from numba import cuda
import math
import time

## Scan - sum

In [2]:
@cuda.jit
def scanBlkKernel(in_arr, n, out_arr, blkSums):
  # transfer data from GMEM to SMEM
  s_data = cuda.shared.array(0,dtype=np.float32)
  i1 = cuda.blockIdx.x * 2 * cuda.blockDim.x + cuda.threadIdx.x
  i2 = i1 + cuda.blockDim.x

  if(i1 < n):
    s_data[cuda.threadIdx.x] = in_arr[i1]
  if i2 < n:
    s_data[cuda.threadIdx.x + cuda.blockDim.x] = in_arr[i2]
  cuda.syncthreads()

  # scan on smem
  # reduction phase
  stride = 1
  while stride < 2 * cuda.blockDim.x:
    s_dataIdx = (cuda.threadIdx.x + 1) * 2 * stride - 1
    if s_dataIdx < 2 * cuda.blockDim.x:
      s_data[s_dataIdx] = s_data[s_dataIdx] + s_data[s_dataIdx - stride]
    stride *= 2
    cuda.syncthreads()

  # post reduction phase
  stride = cuda.blockDim.x // 2
  while stride > 0:
    s_dataIdx = (cuda.threadIdx.x + 1) * 2 * stride - 1 + stride
    if s_dataIdx < 2 * cuda.blockDim.x:
      s_data[s_dataIdx] = s_data[s_dataIdx] + s_data[s_dataIdx - stride]
    stride = stride // 2
    cuda.syncthreads()
    

  # write results from smem to gmem
  if i1 < n:
    out_arr[i1] = s_data[cuda.threadIdx.x]
  if i2 < n:
    out_arr[i2] = s_data[cuda.threadIdx.x + cuda.blockDim.x]

  if (blkSums is not None) and (cuda.threadIdx.x == 0):
    blkSums[cuda.blockIdx.x] = s_data[2 * cuda.blockDim.x - 1]
    

In [3]:
@cuda.jit
def addPrevBlk(blkSumsScan, blkScans, n):
  i = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x + cuda.blockDim.x
  if i < n:
    blkScans[i] = blkScans[i] + blkSumsScan[cuda.blockIdx.x];

In [4]:
def scan(in_arr, n , out_arr, use_device=0,blkSize=1):
  if use_device == 0:
    print("Scan by host")
    out_arr[0] = in_arr[0]
    for i in range(1,n):
      out_arr[i] = out_arr[i - 1] + in_arr[i]  
  else:
    blkDataSize = 2 * blkSize
    d_in_arr = cuda.to_device(in_arr)
    d_out_arr = cuda.device_array(n)

    gridSize = math.ceil(n / blkDataSize)
    if gridSize > 1:
      d_blkSums = cuda.device_array(gridSize)
    else:
      d_blkSums = None
    
    smem = blkDataSize * 4
    scanBlkKernel[gridSize,blkSize,0,smem](d_in_arr,n,d_out_arr,d_blkSums)
    cuda.synchronize()

    if gridSize > 1:
      temp = gridSize
      blkSums = d_blkSums.copy_to_host()
      for i in range(1,gridSize):
        blkSums[i] += blkSums[i-1]
      d_blkSums = cuda.to_device(blkSums)
      addPrevBlk[gridSize - 1, blkDataSize](d_blkSums,d_out_arr,n)
      cuda.synchronize()
      return d_out_arr.copy_to_host()



In [5]:
n = 480000
in_arr = np.random.rand(n)

blockSize = 256
correctOut = np.zeros(n)
start = time.time()
scan(in_arr,n,correctOut,0)
end = time.time()
print(f"Processing time: {end - start} s")

Scan by host
Processing time: 0.13895606994628906 s


In [18]:
out = np.zeros(n)

start = time.time()
out = scan(in_arr,n,out,1,256)
end = time.time()
print(f"Processing time: {end - start} s")

Processing time: 0.005572319030761719 s


In [7]:
np.abs(np.mean(out - correctOut))

0.0003072959762989259

## Scan - max

In [8]:
@cuda.jit
def scan_maxBlkKernel(in_arr, n, out_arr, blkSums):
  # transfer data from GMEM to SMEM
  s_data = cuda.shared.array(0,dtype=np.float32)
  i1 = cuda.blockIdx.x * 2 * cuda.blockDim.x + cuda.threadIdx.x
  i2 = i1 + cuda.blockDim.x

  if(i1 < n):
    s_data[cuda.threadIdx.x] = in_arr[i1]
  if i2 < n:
    s_data[cuda.threadIdx.x + cuda.blockDim.x] = in_arr[i2]
  cuda.syncthreads()

  # scan on smem
  # reduction phase
  stride = 1
  while stride < 2 * cuda.blockDim.x:
    s_dataIdx = (cuda.threadIdx.x + 1) * 2 * stride - 1
    if s_dataIdx < 2 * cuda.blockDim.x:
      s_data[s_dataIdx] = max(s_data[s_dataIdx],s_data[s_dataIdx - stride])
    stride *= 2
    cuda.syncthreads()

  # post reduction phase
  stride = cuda.blockDim.x // 2
  while stride > 0:
    s_dataIdx = (cuda.threadIdx.x + 1) * 2 * stride - 1 + stride
    if s_dataIdx < 2 * cuda.blockDim.x:
      s_data[s_dataIdx] = max(s_data[s_dataIdx], s_data[s_dataIdx - stride])
    stride = stride // 2
    cuda.syncthreads()
    

  # write results from smem to gmem
  if i1 < n:
    out_arr[i1] = s_data[cuda.threadIdx.x]
  if i2 < n:
    out_arr[i2] = s_data[cuda.threadIdx.x + cuda.blockDim.x]

  if (blkSums is not None) and (cuda.threadIdx.x == 0):
    blkSums[cuda.blockIdx.x] = s_data[2 * cuda.blockDim.x - 1]
    

In [9]:
@cuda.jit
def maxPrevBlk(blkSumsScan, blkScans, n):
  i = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x + cuda.blockDim.x
  if i < n:
    blkScans[i] = max(blkScans[i],blkSumsScan[cuda.blockIdx.x])

In [10]:
def scan_max(in_arr, n , out_arr, use_device=0,blkSize=1):
  if use_device == 0:
    print("Scan max by host")
    out_arr[0] = in_arr[0]
    for i in range(1,n):
      out_arr[i] = max(out_arr[i - 1],in_arr[i])
  else:
    blkDataSize = 2 * blkSize
    d_in_arr = cuda.to_device(in_arr)
    d_out_arr = cuda.device_array(n)

    gridSize = math.ceil(n / blkDataSize)
    if gridSize > 1:
      d_blkSums = cuda.device_array(gridSize)
    else:
      d_blkSums = None
    
    smem = blkDataSize * 4
    scan_maxBlkKernel[gridSize,blkSize,0,smem](d_in_arr,n,d_out_arr,d_blkSums)
    cuda.synchronize()

    if gridSize > 1:
      temp = gridSize
      blkSums = d_blkSums.copy_to_host()
      for i in range(1,gridSize):
        blkSums[i] = max(blkSums[i-1], blkSums[i])
      d_blkSums = cuda.to_device(blkSums)
      maxPrevBlk[gridSize - 1, blkDataSize](d_blkSums,d_out_arr,n)
      cuda.synchronize()
      return d_out_arr.copy_to_host()

In [11]:
n = 480000
in_arr = np.random.rand(n)

blockSize = 256
correctOut = np.zeros(n)
start = time.time()
scan_max(in_arr,n,correctOut,0)
end = time.time()
print(f"Processing time: {end - start} s")

Scan max by host
Processing time: 0.18138837814331055 s


In [19]:
out = np.zeros(n)

start = time.time()
out = scan_max(in_arr,n,out,1,256)
end = time.time()
print("Scan max by device")
print(f"Processing time: {end - start} s")

Scan max by device
Processing time: 0.006231069564819336 s


In [13]:
np.abs(np.mean(out - correctOut))

1.0004327345773899e-08

## Hàm trừ hai mảng

In [14]:
n = 480000
in_arr1 = np.random.rand(n)
in_arr2 = np.random.rand(n)

start = time.time()
correctOut = in_arr1 - in_arr2
end = time.time()
print(f"Processing time: {end - start}")

Processing time: 0.0022614002227783203


In [15]:
@cuda.jit
def subtract_two_arrays_kernel(in1,in2,out,n):
  ids = cuda.grid(1)
  if ids < n:
    out[ids] = in1[ids] - in2[ids]

def subtract_two_arrays_device(in1,in2,n,blockSize=32):
  d_in1 = cuda.to_device(in1)
  d_in2 = cuda.to_device(in2)
  d_out = cuda.device_array(n)

  gridSize = math.ceil(n / blockSize)
  subtract_two_arrays_kernel[gridSize,blockSize](d_in1,d_in2,d_out,n)

  return d_out.copy_to_host()

In [20]:
start = time.time()
out = subtract_two_arrays_device(in_arr1,in_arr2,n)
end = time.time()
print(f"Processing time: {end - start}")

Processing time: 0.010683774948120117


In [17]:
np.abs(np.mean(out - correctOut))

0.0